In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)

print(fetch_data_to)
print(fetch_data_from)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)



2025-03-04 19:01:25,585 INFO: Initializing external client
2025-03-04 19:01:25,585 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:01:26,165 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
2025-03-04 23:01:25.584787+00:00
2025-02-04 00:01:25.584787+00:00
Fetching data from 2025-02-04 00:01:25.584787+00:00 to 2025-03-04 23:01:25.584787+00:00


In [5]:
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.77s) 


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

pickup_hour  pickup_location_id  rides
0      2025-02-04 01:00:00+00:00                   2      0
1      2025-02-04 02:00:00+00:00                   2      0
2      2025-02-04 03:00:00+00:00                   2      0
3      2025-02-04 04:00:00+00:00                   2      0
4      2025-02-04 05:00:00+00:00                   2      0
...                          ...                 ...    ...
174189 2025-03-04 18:00:00+00:00                 263    132
174190 2025-03-04 19:00:00+00:00                 263     99
174191 2025-03-04 20:00:00+00:00                 263     86
174192 2025-03-04 21:00:00+00:00                 263     76
174193 2025-03-04 22:00:00+00:00                 263     58

[174194 rows x 3 columns]

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [9]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [11]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [12]:
features

rides_t-672  rides_t-671  rides_t-670  rides_t-669  rides_t-668  \
0             32           48           92           76            8   
1              0            0            0            0            0   
2              3            0            0            0            0   
3              0            0            0            0            0   
4              3            2            1            0            0   
..           ...          ...          ...          ...          ...   
246            0            0            0            0            0   
247            0            0            0            0            0   
248          243          130          148          572          217   
249            6            5           22           20            5   
250            0            0            0            0            0   

     rides_t-667  rides_t-666  rides_t-665  rides_t-664  rides_t-663  ...  \
0            155           62           60            6           80  ...   
1              0            0            0            0            0  ...   
2              2            0            2            0            1  ...   
3              0            1            0            0            0  ...   
4              0            0            0            1            0  ...   
..           ...          ...          ...          ...          ...  ...   
246            0            0            0            0            1  ...   
247            0            4            0            1            0  ...   
248          231          173          141          160          167  ...   
249           23            2            6            1           14  ...   
250            0            0            0            0            0  ...   

     rides_t-8  rides_t-7  rides_t-6  rides_t-5  rides_t-4  rides_t-3  \
0           13          5         21        120        218        146   
1            0          0          0          0          1          0   
2            1          3          0          2          1          2   
3            0          0          0          0          0          0   
4            0          0          0          0          0          0   
..         ...        ...        ...        ...        ...        ...   
246          0          0          0          1          0          0   
247          0          0          1          0          0          0   
248          0        133        311        225        168        179   
249         15          1          0          4          0         24   
250          0          0          0          0          0          1   

     rides_t-2  rides_t-1  pickup_location_id         pickup_hour  
0          111         82                 263 2025-02-21 14:00:00  
1            0          1                 210 2025-02-26 19:00:00  
2            1          1                 216 2025-02-25 04:00:00  
3            0          0                 147 2025-02-14 01:00:00  
4            0          0                 217 2025-03-04 02:00:00  
..         ...        ...                 ...                 ...  
246          0          0                  67 2025-02-15 05:00:00  
247          1          1                  86 2025-02-25 16:00:00  
248         37          2                 142 2025-02-13 20:00:00  
249          4          9                  74 2025-02-21 21:00:00  
250          0          0                 153 2025-02-25 16:00:00  

[251 rows x 674 columns]

In [13]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 19:02:07,732 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:02:07,735 INFO: Initializing external client
2025-03-04 19:02:07,736 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-03-04 19:02:08,368 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695
Fetching data from 2025-02-04 00:02:07.732455+00:00 to 2025-03-04 23:02:07.732455+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.64s) 


In [14]:
current_date

Timestamp('2025-03-05 00:02:07.732455+0000', tz='Etc/UTC')

In [15]:
features

rides_t-672  rides_t-671  rides_t-670  rides_t-669  rides_t-668  \
0              0            0            0            0            0   
1              0            0            0            1            0   
2              6            2            0            1            1   
3              0            0            0            0            0   
4              0            1            0            0            1   
..           ...          ...          ...          ...          ...   
246            0            0            0            0            0   
247            0            0            0            0            0   
248            3            0            0            2            2   
249            3            1            1            3           19   
250            5            4            5            7           21   

     rides_t-667  rides_t-666  rides_t-665  rides_t-664  rides_t-663  ...  \
0              0            0            0            0            0  ...   
1              1            0            0            0            0  ...   
2              2            1           11            7            3  ...   
3              0            0            0            0            0  ...   
4              2            2            1            2            3  ...   
..           ...          ...          ...          ...          ...  ...   
246            2            0            0            0            0  ...   
247            1            4            2            0            1  ...   
248            0            4            5           16           21  ...   
249           63          126          177          156          102  ...   
250           46          142          111          142          116  ...   

     rides_t-8  rides_t-7  rides_t-6  rides_t-5  rides_t-4  rides_t-3  \
0            0          0          0          0          0          0   
1            0          0          0          0          0          0   
2            1          1          0          0          1          1   
3            0          0          0          0          0          0   
4            4          1          5          2          1          1   
..         ...        ...        ...        ...        ...        ...   
246          0          0          0          0          0          0   
247          6          1          1          0          0          0   
248         43         45         24         22         21          8   
249         82        103         65         37         16         14   
250        114        116        102         87         72         45   

     rides_t-2  rides_t-1  pickup_location_id         pickup_hour  
0            0          0                   2 2025-03-04 01:00:00  
1            0          0                   3 2025-03-04 01:00:00  
2            1          4                   4 2025-03-04 01:00:00  
3            0          0                   6 2025-03-04 01:00:00  
4            0          0                   7 2025-03-04 01:00:00  
..         ...        ...                 ...                 ...  
246          0          0                 259 2025-03-04 01:00:00  
247          1          0                 260 2025-03-04 01:00:00  
248         10          1                 261 2025-03-04 01:00:00  
249          9          2                 262 2025-03-04 01:00:00  
250         30          5                 263 2025-03-04 01:00:00  

[251 rows x 674 columns]

In [16]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 19:02:21,451 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 19:02:21,472 INFO: Initializing external client
2025-03-04 19:02:21,472 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 19:02:22,021 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214695


In [17]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [18]:
predictions

pickup_location_id  predicted_demand
0                     2               0.0
1                     3               0.0
2                     4               2.0
3                     6               0.0
4                     7               1.0
..                  ...               ...
246                 259               0.0
247                 260               0.0
248                 261               1.0
249                 262               1.0
250                 263               4.0

[251 rows x 2 columns]

In [19]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138, 249,  48, 230, 186, 114,  79, 142,  68], dtype=int32)